In [54]:
##RAG pipeline With vector Database

In [55]:
## Data Imgestion
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain , SequentialChain
from langchain.chains import SimpleSequentialChain
from langchain_community.document_loaders import TextLoader

loader=TextLoader("../requirements.txt")
document=loader.load()
document

[Document(metadata={'source': '../requirements.txt'}, page_content='groq\nlangchain\nstreamlit\npython-dotenv\npyPDF2\n\n-e .')]

In [56]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv()
API_KEY = os.getenv("API_KEY")
print("-------->",API_KEY)
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY") 


RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

--------> 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


In [57]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.docstore.document import Document
import os 

text = """LangChain is a framework for developing applications powered by large language models (LLMs). 
It provides tools to manage, query, and retrieve relevant data from documents efficiently."""

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks=text_splitter.split_text(text)
chunk_documents = [Document(page_content=chunk) for chunk in chunks]
print("-------------->",chunk_documents)



--------------> [Document(metadata={}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs). \nIt provides tools to manage, query, and retrieve relevant data from documents efficiently.')]


In [58]:
from groq import Groq
from langchain_groq import ChatGroq
from dotenv import load_dotenv 
import os
import json
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain , SequentialChain


load_dotenv()

API_KEY = os.getenv("API_KEY")

client = ChatGroq(api_key=API_KEY, model="llama3-70b-8192")


clients = Groq(api_key=API_KEY)


TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}
"""


with open("../src/Response.json","r") as f:
    RESPONSE_JSON=json.load(f)
    
    
    
quiz_generation_prompt=ChatPromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    messages=[ 
        {"role": "user", "content": TEMPLATE}
    ]
)


quiz_chain=LLMChain(llm=client,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

     
     
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


quiz_evaluation_prompt=ChatPromptTemplate(
    input_variables=["subject","quiz"],
    messages=[ 
        {"role": "user", "content": TEMPLATE2}
    ]
)

review_chain=LLMChain(llm=client,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain] , input_variables=["text","number","subject","tone","RESPONSE_JSON"] , output_variables=["quiz","review"],verbose=True)

PATH="../src/data.txt"

with open(PATH,"r") as file:
   TEXT = file.read()
   
   
generate_evaluate_chain

TEXT
NUMBER=20
SUBJECT="AI"
TONE="Simple"
RESPONSE_JSON=RESPONSE_JSON
 

def call_groq(prompt):
    response = clients.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are an expert MCQ creator."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=3000
    )
    return response.choices[0].message.content



def generate_quiz_prompt(text, number, subject, tone, response_json):
    return TEMPLATE.format(
        text=text,
        number=number,
        subject=subject,
        tone=tone,
        RESPONSE_JSON=json.dumps(response_json)
    ) 

quiz_prompt = generate_quiz_prompt(TEXT, NUMBER, SUBJECT, TONE, RESPONSE_JSON)
mcq_response = call_groq(quiz_prompt) 

print(mcq_response)




{"1": {"mcq": "What is data science?", "options": {"a": "A field that only uses statistics", "b": "An interdisciplinary field that uses statistics, computing, and domain knowledge", "c": "A field that only uses computer science and information science", "d": "A field that only uses mathematics and statistics"}, "correct": "b"}, "2": {"mcq": "What does data science integrate from the underlying application domain?", "options": {"a": "Only natural sciences", "b": "Domain knowledge from natural sciences, information technology, and medicine", "c": "Only information technology", "d": "Only medicine"}, "correct": "b"}, "3": {"mcq": "What is data science described as?", "options": {"a": "Only a science", "b": "A science, a research paradigm, a research method, a discipline, a workflow, and a profession", "c": "Only a research method", "d": "Only a discipline"}, "correct": "b"}, "4": {"mcq": "Who imagined data science as a 'fourth paradigm' of science?", "options": {"a": "Jim Gray", "b": "Nat

In [59]:
from datetime import datetime

LOG_FILE="{datetime.now().strftime('%m_%d_%y_%H_%M_%S')}.log"
print(LOG_FILE)


{datetime.now().strftime('%m_%d_%y_%H_%M_%S')}.log


In [60]:
from langchain.document_loaders import TextLoader 

load=TextLoader("../requirements.txt")
hghghhg=load.load()
print("load--------------->",hghghhg)

load---------------> [Document(metadata={'source': '../requirements.txt'}, page_content='groq\nlangchain\nstreamlit\npython-dotenv\npyPDF2\n\n-e .')]


In [61]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

## load,chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://www.grammarly.com/blog/parts-of-speech/articles/#9",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()
text_documents

[Document(metadata={'source': 'https://www.grammarly.com/blog/parts-of-speech/articles/#9'}, page_content='')]

In [62]:
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv
import os

load_dotenv()
GROQ_API_KEY = os.getenv("API_KEY") 
file_path = "C:/Users/HP/Downloads/1.3 Python as a Language.pdf"

loader = PyPDFLoader(file_path)
pages = loader.load()

print("----------->",pages)

-----------> [Document(metadata={'producer': 'Adobe PDF Library 23.8.246', 'creator': 'Acrobat PDFMaker 23 for PowerPoint', 'creationdate': '2024-02-08T13:10:52-05:00', 'moddate': '2024-02-08T13:12:08-05:00', 'title': '1.3 Python as a Language', 'source': 'C:/Users/HP/Downloads/1.3 Python as a Language.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1'}, page_content='Introduction – Part 3 PYTHON FOR\nEVERYBODY\nParseltongue is the language of serpents \nand those who can converse with them.\nAn individual who can speak Parseltongue \nis known as a Parselmouth. It is a very \nuncommon skill, and may be hereditary. \nNearly all known Parselmouths are \ndescended from Salazar Slytherin.\nhttp://harrypotter.wikia.com/wiki/Parseltongue'), Document(metadata={'producer': 'Adobe PDF Library 23.8.246', 'creator': 'Acrobat PDFMaker 23 for PowerPoint', 'creationdate': '2024-02-08T13:10:52-05:00', 'moddate': '2024-02-08T13:12:08-05:00', 'title': '1.3 Python as a Language', 'source': 'C:/Users/H

In [ ]:
from langchain_core.prompts import ChatPromptTemplate 
from langchain.chains.combine_documents import create_stuff_documents_chain
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq


load_dotenv()

API_KEY = os.getenv("API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
client = ChatGroq(api_key=API_KEY,model="llama3-70b-8192") 


prompt = ChatPromptTemplate.from_messages(
    [("system", "Provide me ar_csoc tabel fields :\n\n{context}")]
)

chain = create_stuff_documents_chain(client, prompt)

file_path = "C:/Users/HP/Downloads/soc_coe_db_design_19 (1) (1) (1) (1) (1).pdf"

loader = PyPDFLoader(file_path)
pages = loader.load()

chain.invoke({"context": pages})




invalid pdf header: b'-----'
incorrect startxref pointer(1)
parsing for Object Streams


'The `ar_csoc` table fields are:\n\n1. `id` - uuid, not null\n2. `analysis_review_id` - uuid, not null\n3. `csoc_id` - uuid, not null\n4. `relevant_to_user_entity` - relevant_to_user_entity enum, not null\n5. `user_entity_eval` - text\n6. `status` - ar_item_status enum, not null\n7. `reviewer_client_user_id` - uuid\n8. `approver_client_user_id` - uuid\n9. `reviewed_at` - timestamp without time zone, not null\n10. `approved_at` - timestamp without time zone'